In [66]:
import tkinter as tk
import customtkinter
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog
from tkinter.filedialog import askopenfile
import cv2
import os
import math
from sklearn import metrics
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score,recall_score,precision_score
import numpy as np
import pandas as pd
import time
import random
from sklearn import svm, metrics, datasets
from sklearn.model_selection import GridSearchCV
import csv
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img

import os
import pickle

# --------------------------------Importing and Loading Prepared Models--------------------------------

cnn_model_file = 'cnn_model2.sav'
svm_model_file='svm_model.sav'

cnn_loaded_model = pickle.load(open(cnn_model_file, 'rb'))
svm_loaded_model=pickle.load(open(svm_model_file,'rb'))


def adaptive_histogram_function2(inputImage,enhancementLevel):
    clahe = cv2.createCLAHE(clipLimit = enhancementLevel)
    outputImage = clahe.apply(inputImage)
    return outputImage

def extracting_images(paths):

    images = []
    for path in paths:
        image = load_img(path, target_size=(200, 200))
        image = Image.fromarray(np.uint8(image))
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)


# -------------------------------CNN Report-----------------------------------

testing_folder = 'Testing'



testing_images_paths = []
testing_images_target = []


for label in os.listdir(testing_folder):
    for image in os.listdir(testing_folder+ '/' + label):
        testing_images_paths.append(testing_folder +'/'+label + '/'+image)
        if(label=='YES'):
            testing_images_target.append(1)
        elif(label=='NO'):
            testing_images_target.append(0)



testpredictions=cnn_loaded_model.predict(extracting_images(testing_images_paths))
CNN_Model_Test_Prediction=[]

for i in range(0,len(testpredictions)):
    if testpredictions[i]>=0.5:
        CNN_Model_Test_Prediction.append(1)
    else:
        CNN_Model_Test_Prediction.append(0)



        
        
# -------------------------------SVM Report-----------------------------------        

brain_test_dataset=pd.read_csv('test_set.csv')

#SVM
brain_test_features=np.array(brain_test_dataset.iloc[:,1:16129])
brain_test_target=np.array(brain_test_dataset.iloc[:,0])

SVM_Model_Test_Prediction = svm_loaded_model.predict(brain_test_features)


print("Done")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-12-25 21:27:00         3246
metadata.json                                  2022-12-25 21:27:00           64
variables.h5                                   2022-12-25 21:27:00    127507000
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2

MemoryError: Unable to allocate 1.17 GiB for an array with shape (1311, 200, 200, 3) and data type float64

In [82]:
# --------------------------------------------INTERFACE AND PREDICTIONS------------------------------------------

import tkinter as tk
import customtkinter
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog
from tkinter.filedialog import askopenfile
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.utils import shuffle
from skimage.feature import hog


customtkinter.set_appearance_mode("light")  
customtkinter.set_default_color_theme("blue") 


app = customtkinter.CTk()
app.columnconfigure(0, weight=1)
app.minsize(900, 750)
app.geometry("900x750")
app.title("ML Project")



def upload_file():
    global img, img_main, filename
    global f_name
    f_types = [('Jpg Files', '*.jpg')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    f_name = os.path.basename(filename)
    img_main = Image.open(filename)
    img_resized = img_main.resize((250, 250)) 
    img = ImageTk.PhotoImage(img_resized)
    btn_img1.config(image = img)

def open_image(image):
    image.show()
    
def button_callback():
    print("Button click", combobox_1.get())


def filtered_image_function(inputImage):
    sobel_horizontal_mask=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    sobel_vertical_mask=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    
    horizontal_edges=cv2.filter2D(inputImage, -1, sobel_horizontal_mask)
    vertical_edges=cv2.filter2D(inputImage, -1, sobel_vertical_mask)
    
    sobel_edges=cv2.add(horizontal_edges, vertical_edges)
    finalImage=cv2.add(sobel_edges, inputImage)

    return finalImage
    
def process_sobel():
    global sobel_img_pil
    cv_img = cv2.imread(filename)
    img_bw = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    sobel_image = filtered_image_function(img_bw)
    sobel_img_pil = Image.fromarray(sobel_image)
    sobel_img_pil.save("filtered-image.jpg")
    #img_enhanced = Image.open("filtered-image.jpg")
    #img_resized = img_enhanced.resize((170,170)) # new width & height
    #img = ImageTk.PhotoImage(img_resized)
    #btn_img2.config(image = img)

    img_resized = sobel_img_pil.resize((230,230))
    s_img = ImageTk.PhotoImage(img_resized)
    btn_img2.config(image = s_img)
    sobel_img_pil.show()


    
def adaptive_histogram_function2(enhancementLevel):
    cv_img = cv2.imread(filename)
    img_bw = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit = enhancementLevel)
    outputImage = clahe.apply(img_bw)
    AH_image = Image.fromarray(outputImage)
    #AH_image.show()
    return outputImage
    
def adaptive_histogram_slider(e):
    enhancement_value = round(AH_slider.get(), 1)
    AH_value_label.configure(text = enhancement_value)
    cv_img = cv2.imread(filename)
    img_bw = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit = enhancement_value)
    outputImage = clahe.apply(img_bw)
    adapt_histo_img = Image.fromarray(outputImage)
    ah_img_resized = adapt_histo_img.resize((230,230))
    ah_img = ImageTk.PhotoImage(ah_img_resized)
    btn_img3.config(image = ah_img)
    adapt_histo_img.update()


def adaptive_histogram_function():
    global adapt_histo_img
    enhancement_value = round(AH_slider.get(), 1)
    cv_img = cv2.imread(filename)
    img_bw = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit = enhancement_value)
    outputImage = clahe.apply(img_bw)
    adapt_histo_img = Image.fromarray(outputImage)
    ah_img_resized = adapt_histo_img.resize((230,230))
    ah_img = ImageTk.PhotoImage(ah_img_resized)
    btn_img3.config(image = ah_img)
    adapt_histo_img.update()
    

def svm_prediction(path):
    
    svm_window = customtkinter.CTkToplevel()
    svm_window.geometry("490x400")
    svm_window.title("SVM Performance Stats")
    topframe_1 = customtkinter.CTkFrame(master=svm_window, corner_radius=15)
    topframe_1.grid(row = 1, column = 0, columnspan = 5, rowspan = 7, pady=20, padx=10, sticky = 'we')
    # create label on CTkToplevel window
    label1 = customtkinter.CTkLabel(master = topframe_1, text="Support Vector Machine ", text_font=("Montserrat", 12, 'bold'))
    label1.grid(row = 0, column = 0, padx=20, pady=5)
    outlabel1 = customtkinter.CTkLabel(master = topframe_1, text="", text_font=("Montserrat", 15, 'bold'))
    outlabel1.grid(row = 1, column = 0, rowspan = 2, padx=30, pady=5)
    
    label2 = customtkinter.CTkLabel(master = topframe_1, text="Classification Report ", text_font=("Montserrat", 13, 'bold'))
    label2.grid(row = 3, column = 0, padx=20, pady=5)
    outlabel2 = customtkinter.CTkLabel(master = topframe_1, text="", text_font=("Montserrat", 14))
    outlabel2.grid(row = 4, column = 0, rowspan = 5, padx=30, pady=5)
    
    
    #image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    output=adaptive_histogram_function2(4.2)
    output=cv2.resize(output,(127,127))
    HOG=hog(output)
    dataframeHOG=pd.DataFrame(HOG)

    input_image_features=np.array(dataframeHOG)


    SVM_input_image_prediction = svm_loaded_model.predict(input_image_features.transpose())
    print("SVM predicts",SVM_input_image_prediction[0])
    #SVM_final_prediction=""
    if SVM_input_image_prediction[0]=="YES":
        SVM_final_prediction="Tumor was detected"
    elif SVM_input_image_prediction[0]=="NO":
        SVM_final_prediction="Tumor was not detected"

    reportsvm=classification_report(brain_test_target, SVM_Model_Test_Prediction)
    print(SVM_final_prediction)
    outlabel1.configure(text = SVM_final_prediction)
    outlabel2.configure(text = reportsvm)
    
    
def cnn_prediction(path):
    
    cnn_window = customtkinter.CTkToplevel()
    cnn_window.geometry("490x400")
    cnn_window.title("CNN Performance Stats")
    topframe_2 = customtkinter.CTkFrame(master=cnn_window, corner_radius=15)
    topframe_2.grid(row = 1, column = 0, columnspan = 5, rowspan = 7, pady=20, padx=10)
    # create label on CTkToplevel window
    label3 = customtkinter.CTkLabel(master = topframe_2, text="Convolution Neural Network ", text_font=("Montserrat", 12, 'bold'))
    label3.grid(row = 0, column = 0, padx=20, pady=5)
    outlabel3 = customtkinter.CTkLabel(master = topframe_2, text="", text_font=("Montserrat", 15, 'bold'))
    outlabel3.grid(row = 1, column = 0, rowspan = 2, padx=30, pady=5)
    
    label4 = customtkinter.CTkLabel(master = topframe_2, text="Classification Report ", text_font=("Montserrat", 13, 'bold'))
    label4.grid(row = 3, column = 0, padx=20, pady=5)
    outlabel4 = customtkinter.CTkLabel(master = topframe_2, text="", text_font=("Montserrat", 14))
    outlabel4.grid(row = 4, column = 0, rowspan = 5, padx=30, pady=5)
    
    predictionimages = [] #The model takes a numpy array
    image = load_img(path, target_size=(200, 200))
    image = Image.fromarray(np.uint8(image))
    image = np.array(image)/255.0
    predictionimages.append(image)
    predictionimages=np.array(predictionimages)


    predictions = cnn_loaded_model.predict(predictionimages)
    if(predictions[0][0]<0.5):
        CNN_final_prediction="Tumor was not detected"
    elif(predictions[0][0]>=0.5):
        CNN_final_prediction="Tumor was detected"
    

    
    reportCNN=classification_report(CNN_Model_Test_Prediction,testing_images_target)
    
    outlabel3.configure(text = CNN_final_prediction)
    outlabel4.configure(text = reportCNN)
    


temp_img=Image.open('temp_img.jpeg')
temp_resized=temp_img.resize((250,250))
temp_photo=ImageTk.PhotoImage(temp_resized)


frame_1 = customtkinter.CTkFrame(master=app, corner_radius=15)
frame_1.grid(row = 0, column = 3, columnspan = 4, rowspan = 1, pady=20, padx=10)


label_disp = customtkinter.CTkLabel(master=frame_1, text="Brain Tumor Detection", text_font=('@Microsoft JhengHei', 20, 'bold'))
label_disp.grid(row = 0, column = 0, columnspan=4, padx=10, pady=10, sticky='we')


frame_2 = customtkinter.CTkFrame(master=app, corner_radius=15)
frame_2.grid(row = 2, column = 0, columnspan = 4, rowspan = 7, pady=20, padx=10, sticky='we')

upload_label = customtkinter.CTkLabel(master = frame_2, text="Upload Image", text_font=("Montserrat", 12, 'bold'))
upload_label.grid(row=0, column = 0, padx=10, pady=20, sticky='we')

btn_img1 =tk.Button(frame_2, image = temp_photo, command = lambda:open_image(img_main)) # using Button 
btn_img1.grid(row=1,column=0,  padx = 50, pady = 30, sticky='we')

upload_button = customtkinter.CTkButton(frame_2, text="Upload File", border_width=2, text_color = ('white'), command = lambda:upload_file())
upload_button.grid(row = 2, column = 0, padx=10, pady=30, sticky='s')


frame_3 = customtkinter.CTkFrame(master=app, corner_radius=15)
frame_3.grid(row = 2, column = 4, columnspan = 3, rowspan = 7, pady=20, padx=10, sticky='we')

btn_img3 =tk.Button(frame_3, image = temp_photo, command = lambda:open_image(adapt_histo_img)) # using Button 
btn_img3.grid(row=1,column=0, padx = 50, pady = 10)

AH_label = customtkinter.CTkLabel(master = frame_3, text="Adaptive Histogram Image", text_font=("Montserrat", 12, 'bold'))
AH_label.grid(row=0, column = 0, padx=10, pady=10, sticky='we')

AH_slider = customtkinter.CTkSlider(master = frame_3, from_ = 1, to = 5, number_of_steps = 20, command = adaptive_histogram_slider)
AH_slider.grid(row = 3, column = 0, pady=10, padx=20, sticky="we")
AH_slider.set(1)

AH_value_label = customtkinter.CTkLabel(master = frame_3, text= AH_slider.get(), text_font=("Montserrat", 12, 'bold'))
AH_value_label.grid(row=4, column = 0, columnspan=2, padx=10, pady=10, sticky='we')

button_3 = customtkinter.CTkButton(master=frame_3, text="Adaptive Histogram Equalization", 
                                   border_width=2, text_color = ('white'), command = lambda:adaptive_histogram_function())
button_3.grid(row=2, column=0, pady=10, padx=20, sticky='s')


frame_4 = customtkinter.CTkFrame(master=app, corner_radius=15)
frame_4.grid(row = 2, column = 7, columnspan = 3, rowspan = 7, pady=20, padx=10, sticky='we')

sobel_image_label = customtkinter.CTkLabel(master = frame_4, text="Sobel Image", text_font=("Montserrat", 12, 'bold'))
sobel_image_label.grid(row=0, column = 0, padx=10, pady=20, sticky='we')

btn_img2 =tk.Button(frame_4, image = temp_photo, command = lambda:open_image(sobel_img_pil)) # using Button 
btn_img2.grid(row=1,column=0,  padx = 50, pady = 30)

sobel_button = customtkinter.CTkButton(master=frame_4, text="Sobel Gradient", border_width=2, text_color = ('white'), command = lambda:process_sobel())
sobel_button.grid(row=2, column=0, pady=30, padx=10, sticky='s')



frame_5 = customtkinter.CTkFrame(master=app, corner_radius=15)
frame_5.grid(row = 9, column = 2, columnspan = 5, rowspan = 2, pady=20, padx=0, sticky = 'we')

model_title = customtkinter.CTkLabel(master = frame_5, text="Models", text_font=("Montserrat", 15, 'bold'))
model_title.grid(row=0, column = 0, columnspan=3, padx=10, pady=10, sticky='we')

svm_button = customtkinter.CTkButton(master=frame_5, text="SVM Model Prediction", 
                                   border_width =2, text_color = ('white'), command = lambda:svm_prediction(filename))
svm_button.grid(row=1, column=0, pady=10, padx=10, sticky='s')

cnn_button = customtkinter.CTkButton(master=frame_5, text="CNN Model Prediction", 
                                   border_width =2, text_color = ('white'), command = lambda:cnn_prediction(filename))
cnn_button.grid(row=1, column=2, pady=10, padx=10, sticky='s')


app.mainloop()